In [ ]:
!python3 -m pip install selenium
!python -m pip install supabase
!pip3 install webdriver-manager
!pip install pybettor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.2
    Uninstalling websockets-14.2:
      Successfully uninstalled websockets-14.2


In [ ]:
# Needed libs
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

In [ ]:
import supabase
from supabase import create_client
import json
import pandas as pd
from datetime import date
import statsmodels
import numpy as np
import datetime
from datetime import datetime
import time
import pandas as pd
import numpy as np
import math #The Python math module
from scipy import stats #The SciPy stats module
import time
import signal
import sys
from selenium import webdriver
from selenium.webdriver import ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import webdriver_manager.chrome
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.keys import Keys
from fractions import Fraction
from pybettor import convert_odds
from pybettor import implied_prob
import logging

In [ ]:
def driver_uniBet():
    #options = Options()
    webdriver_options = ChromeOptions()
    service = Service(executable_path=r'YOUR PATH HERE')

    useragentarray = [
        "Mozilla/5.0 (Linux; Android 13) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.5672.76 Mobile Safari/537.36"
    ]

    webdriver_options.add_argument("--disable-blink-features=AutomationControlled")
    webdriver_options.add_argument("--no-sandbox")
    webdriver_options.add_argument("--disable-dev-shm-usage")
    webdriver_options.add_argument("--disable-search-engine-choice-screen")
    webdriver_options.add_argument("--disable-dev-shm-usage")
    webdriver_options.add_argument("--headless")
    #options.add_argument(f"--user-data-dir=./profile{driver_num}")

    webdriver_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    webdriver_options.add_experimental_option("useAutomationExtension", False)
    webdriver_options.add_argument("disable-infobars")
    webdriver_options.add_argument("disable-blink-features=AutomationControlled")

    logging.basicConfig(level=logging.DEBUG)

    service = Service(executable_path=r'/usr/bin/chromedriver')
    service.command_line_args()

    #driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver = webdriver.Chrome(options=webdriver_options)
    #driver = webdriver.Chrome(
    #     ChromeDriverManager().install(),
    #     service=service,
    #     options=options
    #)

    driver.execute_script(
        "Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"
    )

    driver.execute_cdp_cmd(
        "Network.setUserAgentOverride", {"userAgent": useragentarray[0]}
    )

    webdriver_options.add_argument("--disable-popup-blocking")
    #     driver.execute_script(
    #         """setTimeout(() => window.location.href="https://www.bet365.com.au", 100)"""
    #     )

    #url = "https://www.unibet.ie/betting/sports/filter/football/england/premier_league/all/matches"
    #driver.get(url)
    #time.sleep(3)
    return driver

# Automation

In [ ]:
# Define a signal handler for manual interruption (Ctrl+C)
#def signal_handler(sig, frame):
#    print("Manual cancellation received. Stopping scraping.")
#    sys.exit(0)

# Attach the signal handler
#signal.signal(signal.SIGINT, signal_handler)


# Global variable to store previous odds for tracking movements
previous_odds = {}

'''
INSERT THE URL HERE
'''

url = "https://www.unibet.com/betting/sports/event/1020895790"



# Scraping function
def scrape_unibet():
    global previous_odds

    try:
        driver = driver_uniBet()
        # Open the URL
        driver.get(url)
        time.sleep(5)  # Allow time for the page to load

        # Handle modal popup
        try:
            buttons = driver.find_elements(By.XPATH, "//div[contains(@class, 'modal')]/button[2]")
            if buttons:
                buttons[0].click()
                time.sleep(2)  # Allow time for modal to close
        except Exception as e:
            print(f"Error interacting with modal: {e}")


        # Scrape game information
        racesFinal, horsesFinal, oddsFinal, selectionsFinal = [], [], [], []


        '''
        SCRAPING UNIBET
        '''

        try:
            # Locate game containers
            games = driver.find_elements(By.XPATH, "//div[contains(@class, 'df7bb')]")
            lengthGames = min(len(games), 10)  # Limit to 10 games or fewer
            print(f"Number of games found: {lengthGames}")

            # Iterate through games to extract information
            for game in games[:lengthGames]:
                try:
                    # Extract race names
                    races = driver.find_elements(By.XPATH, "//div[contains(@class, '_89d76')]")
                    racesFinal.append([race.text for race in races])

                    # Extract horse names
                    horses = driver.find_elements(By.XPATH, "//div[contains(@class, 'KambiBC-outcomes-list__column')]")
                    horsesFinal.append([horse.text for horse in horses])

                    # Extract odds
                    odds = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-iGgWBj kAIwQy')]")
                    oddsFinal.append([odd.text for odd in odds])

                    # Extract selection names or other relevant information
                    selections = driver.find_elements(By.XPATH, "//div[contains(@class, 'KambiBC-outcomes-list__label-main')]")
                    selectionsFinal.append([selection.text for selection in selections])
                except Exception as e:
                    print(f"Error extracting data for a game: {e}")
        except Exception as e:
            print(f"Error finding games: {e}")
            lengthGames = 0

        # Print the scraped data
        print("Matches:", racesFinal)
        print("Horses:", horsesFinal)
        print("Odds:", oddsFinal)
        print("Markets:", selectionsFinal)



        '''
        DATA PROCESSING

        Expected Output
          Player Winner  1-3
        0  Horse1    1.5  1.2
        1  Horse2    2.0  1.8
        2  Horse3    2.5  2.0

        '''


        # Extract data from the nested structure
        data = horsesFinal[0] if horsesFinal else []
        if not data:
          print("No data found.")
          return

        horse_names = data[0].split('\n') if data else []
        winner_odds = data[1].split('\n') if len(data) > 1 else []
        place_odds = data[2].split('\n')   if len(data) > 2 else []
        #horse_names = data[0].split('\n')[1:] if data else []
        #winner_odds = data[1].split('\n')[1:] if len(data) > 1 else []
        #place_odds = data[2].split('\n')[1:]   if len(data) > 2 else []

        # Helper function to split leading spaces and odds values
        def split_leading_space(item):
            if isinstance(item, str):
                match = re.match(r'(\s*)(\d+(\.\d+)?)', item)
                if match:
                    return [match.group(1), match.group(2)]  # Return leading space and number
                return [item]  # Return item unchanged if no match
            return [item]  # Return item unchanged if not a string

        # Apply the function to each item in the list
        winner_split_odds = [split_leading_space(odds) for odds in winner_odds]
        place_split_odds = [split_leading_space(odds) for odds in place_odds]

        # Flatten the list of lists
        winner_flat_odds = [item for sublist in winner_split_odds for item in sublist if item != ''][1:]
        place_flat_odds = [item for sublist in place_split_odds for item in sublist if item != ''][1:]

        # Combine data into a structured list
        combined_data = []
        movements = []

        for i in range(len(horse_names)):
            player_name = horse_names[i].strip()
            winner = winner_flat_odds[i].strip() if i < len(winner_flat_odds) else "N/A"
            place = place_flat_odds[i].strip() if i < len(place_flat_odds) else "N/A"

            if winner and winner.replace('.', '', 1).isdigit():
                winner = float(winner)
            else:
                winner = float('inf')


            combined_data.append([player_name, winner])


        # Create DataFrame for combined data
        df = pd.DataFrame(combined_data, columns=["Player", "Winner"])
        print("\nCombined DataFrame:")
        print(df)


        '''
        MOVEMENT DETECTION

        Expect Output:
        Odds Movements Detected:
          Horse  Previous Odd  New Odd
        0  Horse A          2.0      1.8
        2  Horse C          4.0      3.9

        '''
        for _, row in df.iterrows():
            player_name = row['Player']
            winner = row['Winner']

            # Initialize odds for new players
            if player_name not in previous_odds:
                # If the horse is not yet tracked, initialize its odds
                previous_odds[player_name] = winner
            else:
                # Track odds movements
                prev_odd = previous_odds[player_name]

                if prev_odds is not None and winner < prev_odds:
                  movements.append({
                      "Horse": player_name,
                      "Previous Odds": prev_odds,
                      "New Odds": winner
                  })

            # Update the previous odds
            previous_odds[player_name] = winner


        # Print odds movements
        if movements:
            print("\nOdds Movements:")
            df_movements = pd.DataFrame(movements)
            print(df_movements)
        else:
            print("\nNo odds movements detected.")
            #print(f"Updated Previous Odds: \n{previous_odds}")

    finally:
        # Close the driver
        driver.quit()
        print("\nWebDriver closed.")


# Periodic scraping loop
if __name__ == "__main__":
    print("Starting periodic scraping. Press Stop in Jupyter Kernel or Interrupt Execution to stop the loop..")

    while True:
        try:
            # Call the scraping function
            scrape_unibet()


            # Wait for 15 minutes before the next iteration
            print("Waiting for 15 minutes...")
            time.sleep(3*60)
        except KeyboardInterrupt:
            print("Scraping stopped manually.")
            break
        except Exception as e:
            print(f"Error during periodic scraping: {e}")

Starting periodic scraping. Press Stop in Jupyter Kernel or Interrupt Execution to stop the loop..
Number of games found: 1
Matches: [['Elitloppet 2025']]
Horses: [[' Idao De Tillard\nFrancesco Zet\nHorsy Dream\nBorups Victory\nGo On Boy\nSan Moteur\nHohneck\nHooker Berry', 'Winner\n4.50\n6.00\n6.00\n8.00\n8.00\n8.00\n16.00\n25.00', '(1-3)\n2.35\n2.55\n2.75\n2.85\n3.15\n3.15\n5.00\n8.00']]
Odds: [['4.50', '6.00', '6.00', '8.00', '8.00', '8.00', '16.00', '25.00', '2.35', '2.55', '2.75', '2.85', '3.15', '3.15', '5.00', '8.00']]
Markets: [['Idao De Tillard', 'Francesco Zet', 'Horsy Dream', 'Borups Victory', 'Go On Boy', 'San Moteur', 'Hohneck', 'Hooker Berry']]

Combined DataFrame:
            Player  Winner
0  Idao De Tillard     4.5
1    Francesco Zet     6.0
2      Horsy Dream     6.0
3   Borups Victory     8.0
4        Go On Boy     8.0
5       San Moteur     8.0
6          Hohneck    16.0
7     Hooker Berry    25.0

No odds movements detected.

WebDriver closed.
Waiting for 15 minute

# Manual Checking

In [ ]:
try:
    driver = driver_uniBet()

    '''
    INSERT THE URL HERE
    '''
    url = "https://www.unibet.com/betting/sports/event/1020895790"


    # Open the URL
    driver.get(url)
    time.sleep(5)  # Allow time for page to load

    # Handle modal popup
    try:
        buttons = driver.find_elements(By.XPATH, "//div[contains(@class, 'modal')]/button[2]")
        if buttons:
            buttons[0].click()
            time.sleep(2)  # Allow time for modal to close
    except Exception as e:
        print(f"Error interacting with modal: {e}")


    '''
    SCRAPING UNIBET
    '''

    # Scrape game information
    racesFinal = []
    horsesFinal = []
    oddsFinal = []
    selectionsFinal = []

    try:
        # Locate game containers
        #games = driver.find_elements(By.CSS_SELECTOR, ".fc699")  # Update selector as per the site's structure
        #ames = driver.find_elements(By.XPATH, "//div[contains(@class, '_375d1 ad8ca')]")
        #games = driver.find_elements(By.XPATH, "//div[contains(@class, 'fc699')]")
        games = driver.find_elements(By.XPATH, "//div[contains(@class, 'df7bb')]")
        lengthGames = min(len(games), 10)  # Limit to 10 games or fewer
        print(f"Number of games found: {lengthGames}")

        # Iterate through games to extract information
        for game in games[:lengthGames]:
            try:
                # Extract race names
                #players = game.find_elements(By.CSS_SELECTOR, ".outrightsSection")
                races = driver.find_elements(By.XPATH, "//div[contains(@class, '_89d76')]")
                racesFinal.append([race.text for race in races])

                # Extract horses names
                #horses = game.find_elements(By.CSS_SELECTOR, ".outrightsSection")
                horses = driver.find_elements(By.XPATH, "//div[contains(@class, 'KambiBC-outcomes-list__column')]")
                horsesFinal.append([horse.text for horse in horses])

                # Extract odds
                #odds = game.find_elements(By.CSS_SELECTOR, ".outrightSectionData")
                #odds = driver.find_elements(By.XPATH, "//div[contains(@class, '_67716')]")
                odds = driver.find_elements(By.XPATH, "//div[contains(@class, 'sc-iGgWBj kAIwQy')]")
                oddsFinal.append([odd.text for odd in odds])

                # Extract selections names or other relevant information
                #selections = game.find_elements(By.CSS_SELECTOR, ".sectionHeader")
                #selections = driver.find_elements(By.XPATH, "//div[contains(@class, '_4ec65 aabf1')]")
                selections = driver.find_elements(By.XPATH, "//div[contains(@class, 'KambiBC-outcomes-list__label-main')]")
                selectionsFinal.append([selection.text for selection in selections])
            except Exception as e:
                print(f"Error extracting data for a game: {e}")

    except Exception as e:
        print(f"Error finding games: {e}")
        lengthGames = 0

    # Print the scraped data
    print("Matches:", racesFinal)
    print("Horses:", horsesFinal)
    print("Odds:", oddsFinal)
    print("Markets:", selectionsFinal)


    '''
    DATA PROCESSING

    Expected Output
       Player Winner  1-3
    0  Horse1    1.5  1.2
    1  Horse2    2.0  1.8
    2  Horse3    2.5  2.0

    '''

    # Extract data from the nested structure
    data = horsesFinal[0] if horsesFinal else []
    if not data:
        print("No data found.")

    horse_names = data[0].split('\n')[1:] if data else []
    winner_odds = data[1].split('\n')[1:] if len(data) > 1 else []
    place_odds = data[2].split('\n')[1:]   if len(data) > 2 else []


    def split_leading_space(item):
        if isinstance(item, str):  # Ensure item is a string
            # Check if the string starts with a space, then split into the space and number
            match = re.match(r'(\s*)(\d+(\.\d+)?)', item)
            if match:
                return [match.group(1), match.group(2)]  # Return the leading space and the number separately
            return [item]  # Return the item as-is if no leading space
        else:
            return [item]  # If the item is not a string, return it unchanged

    # Apply the function to each item in the list
    winner_split_odds = [split_leading_space(odds) for odds in winner_odds]
    place_split_odds = [split_leading_space(odds) for odds in place_odds]

    # Flatten the list of lists
    winner_flat_odds = [item for sublist in winner_split_odds for item in sublist if item != ''][1:]
    place_flat_odds = [item for sublist in place_split_odds for item in sublist if item != ''][1:]

    combined_data = []
    for i in range(len(horse_names)):
        player_name = horse_names[i].strip()
        winner = winner_flat_odds[i].strip() if i < len(winner_flat_odds) else "N/A"
        place = place_flat_odds[i].strip() if i < len(place_flat_odds) else "N/A"

        if winner and winner.replace('.', '', 1).isdigit():
            winner = float(winner)
        else:
            winner = float('inf')

        # Ensure the number of additional odds matches the player count
        combined_data.append([player_name, winner, place])

    # Create DataFrame
    df = pd.DataFrame(combined_data, columns=["Player", "Winner", "1-3"])
    print("\nCombined DataFrame:")
    print(df)



    '''
    MOVEMENT DETECTION

    Expect Output:
    Odds Movements Detected:
       Horse  Previous Odd  New Odd
    0  Horse A          2.0      1.8
    2  Horse C          4.0      3.9

    '''

    previous_odds = {}
    movements = []


    # Track odds movements
    prev_odds = previous_odds.get(player_name)
    if prev_odds is not None and winner < prev_odds:
        movements.append({
            "Horse": player_name,
            "Previous Odds": prev_odds,
            "New Odds": winner
        })

    # Update the previous odds
    previous_odds[player_name] = winner


    # Print odds movements
    if movements:
        print("\nOdds Movements:")
        df_movements = pd.DataFrame(movements)
        print(df_movements)
    else:
        print("\nNo odds movements detected.")

finally:
    # Close the driver
    driver.quit()
    print("WebDriver closed.")

Number of games found: 1
Matches: [['250128 - Eskilstuna # 9']]
Horses: [[" 1640a\n1Lexington Yoda\n2Valnes Isidor\n3Billy The Bronco\n4Bombibitt S.I.R.\n5Solkattens Eros\n6Otomaro Zon\n7Herr Melin\n8Belgique\n9He's From Heaven\n10Elysia\n11Timo Jutila\n12B.W.L.Fighter", 'Winner\n 2.85\n250.0\n2.95\n65.00\n250.0\n8.00\n85.00\n11.00\n4.15\n45.00\n 75.00', '(1-3)\n 1.35\n20.00\n1.50\n6.50\n25.00\n1.75\n9.00\n2.55\n1.55\n6.00\n 8.00', "Lexington Yoda\n1.60\nHe's From Heaven\n2.20\n Billy The Bronco\n1.55\nOtomaro Zon\n2.30\n Belgique\n1.70\nElysia\n4.25\nBombibitt S.I.R.\n3.50"]]
Odds: [['2.85', '250.0', '2.95', '65.00', '250.0', '8.00', '85.00', '11.00', '4.15', '45.00', '75.00', '1.35', '20.00', '1.50', '6.50', '25.00', '1.75', '9.00', '2.55', '1.55', '6.00', '8.00', '1.60', '2.20', '1.55', '2.30', '1.70', '4.25', '3.50']]
Markets: [['1640a', '1Lexington Yoda', '2Valnes Isidor', '3Billy The Bronco', '4Bombibitt S.I.R.', '5Solkattens Eros', '6Otomaro Zon', '7Herr Melin', '8Belgique', "9H

In [ ]:
print(winner)
print(previous_odds)
print(player_name)
print(prev_odds)
print(winner)



75.0
{'12B.W.L.Fighter': 75.0}
12B.W.L.Fighter
None
75.0


In [ ]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

https://docs.google.com/spreadsheets/d/1i-hbMxet5JgQLUZR264BGex1LwafXoo749UITuUYUDc#gid=0


In [ ]:
# Flatten the list and extract relevant data
#data = horsesFinal[0]  # since it's nested
#horse_names = data[0].split('\n')[1:]  # Skip first element and get the player names
#winner_odds = data[1].split('\n')[1:]  # Skip first element and get the winner odds
#place_odds = data[2].split('\n')[1:]  # Skip first element and get the winner odds

# Print lengths and data to debug
#print(f"Horse Names: {len(horse_names)}")
#print(f"Winner Odds: {len(winner_odds)}")
#print(f"1-3 Odds: {len(place_odds)}")

#print(horse_names)
#print(winner_odds)
#print(place_odds)

# All Competitions

In [ ]:
driver = driver_uniBet()

# Open the URL
url = "https://www.unibet.com/betting/sports/filter/trotting/sweden/all/competitions"
driver.get(url)
time.sleep(5)  # Allow time for page to load

# Handle modal popup
try:
    buttons = driver.find_elements(By.XPATH, "//div[contains(@class, 'modal')]/button[2]")
    if buttons:
        buttons[0].click()
        time.sleep(2)  # Allow time for modal to close
except Exception as e:
    print(f"Error interacting with modal: {e}")

# Scrape game information
racesFinal = []
oddsFinal = []
outrightsFinal = []

try:
    # Locate game containers
    #games = driver.find_elements(By.CSS_SELECTOR, "._7d30d _8b0e1")  # Update selector as per the site's structure
    games = driver.find_elements(By.XPATH, "//div[contains(@class, '_7d30d _8b0e1')]")
    lengthGames = min(len(games), 10)  # Limit to 10 games or fewer
    print(f"Number of games found: {lengthGames}")

    # Iterate through games to extract information
    for game in games[:lengthGames]:
        try:
            # Extract outrights names or other relevant information
            #outrights = game.find_elements(By.CSS_SELECTOR, ".sectionHeader")  # Replace with correct selector
            #outrights = driver.find_elements(By.XPATH, "//div[contains(@class, '_4ec65 aabf1')]")
            outrights = driver.find_elements(By.XPATH, "//div[contains(@class, '_488fa _498cc fd7df')]")
            outrightsFinal.append([outright.text for outright in outrights])

            # Extract race names
            #races = game.find_elements(By.CSS_SELECTOR, ".outrightsSection")  # Replace with correct selector
            races = driver.find_elements(By.XPATH, "//div[contains(@class, '_2b281')]")
            #races = driver.find_elements(By.XPATH, "//div[contains(@class, 'cb228 _6361c')]")
            racesFinal.append([race.text for race in races])

            # Extract odds
            #odds = game.find_elements(By.CSS_SELECTOR, ".outrightSectionData")  # Replace with correct selector
            #odds = driver.find_elements(By.XPATH, "//div[contains(@class, '_67716')]")
            odds = driver.find_elements(By.XPATH, "//div[contains(@class, '_8dc1f')]")
            oddsFinal.append([odd.text for odd in odds])

        except Exception as e:
            print(f"Error extracting data for a game: {e}")

except Exception as e:
    print(f"Error finding games: {e}")
    lengthGames = 0

# Print the scraped data
print("Outrights:", outrightsFinal)
print("Races:", racesFinal)
print("Odds:", oddsFinal)

# Close the driver
driver.quit()


Number of games found: 1
Outrights: [['Elitloppet 2025\n25 May 2025 15:45']]
Races: [['Elitloppet 2025\n25 May 2025 15:45\nHorsy Dream\n5.00\nFrancesco Zet\n6.00\nIdao De Tillard\n7.50\nBorups Victory\n8.00\nSan Moteur\n9.00\nView all']]
Odds: [['Horsy Dream\n5.00\nFrancesco Zet\n6.00\nIdao De Tillard\n7.50\nBorups Victory\n8.00\nSan Moteur\n9.00\nView all']]
